In [1]:
import numpy as np
import pandas as pd
from senticnet5 import senticnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import re
from scipy import sparse
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

In [2]:
df = pd.read_excel('hand8_k_random.xlsx')
print(len(df))

8501


In [3]:
singleword=[]
for key,val in senticnet.items():
    if(len(key.split('_'))==1):
        singleword.append(key)
print(len(singleword))
word=[]
primary=[]
sec=[]
pola=[]
for x in singleword:
    word.append(x)
    primary.append(senticnet[x][4])
    sec.append(senticnet[x][5])
    pola.append(senticnet[x][7])
df_emo=pd.DataFrame(list(zip(word,primary,sec,pola)),columns=["Word","Primary","Secondary","Polarity"])
df_emo.sample(10)

39891


,Word,Primary,Secondary,Polarity
7027,conoidal,#joy,#admiration,0.034
38949,web-based,#joy,#interest,0.787
31365,seismic,#sadness,#fear,-0.78
6868,conductivity,#interest,#admiration,0.881
18631,investor,#interest,#interest,0.076
12865,female,#joy,#admiration,0.836
9120,destabilize,#sadness,#anger,-0.72
11080,electropositive,#joy,#admiration,0.666
24522,ontogeny,#anger,#disgust,-0.75
21990,microcarpa,#joy,#surprise,0.715


In [4]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.multioutput import ClassifierChain
from skmultilearn.ensemble import RakelD

In [5]:
def remove_count_user_mentions(tweet):
    tweet_mentions_removed = re.subn(r'@[A-Za-z0-9]+','',tweet)
    tweet = tweet_mentions_removed[0]
    no_user_mentions = tweet_mentions_removed[1]
    return tweet,no_user_mentions
#%%
def remove_count_urls(tweet):
    tweet_url_removed = re.subn('https?://[A-Za-z0-9./]+','',tweet)
    tweet = tweet_url_removed[0]
    no_urls = tweet_url_removed[1]
    return tweet,no_urls
#%%
def remove_count_hashtags(tweet):
    no_hashtags = len({tag.strip("#") for tag in tweet.split() if tag.startswith("#")})
    tweet = re.sub("[^a-zA-Z]", " ",tweet)
    return tweet,no_hashtags    
def get_pos(word):
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [6]:
need = ["J","N","V","R"]
#need = ["V"]
neg = ["n't","not"]
punct = [".",",","?",";","!"]
opposite = {}
opposite["#joy"] = "#sadness"
opposite[0] = 1
opposite["#sadness"] = "#joy"
opposite[1] = 0
opposite["#admiration"] = "#anger"
opposite[4] = 2
opposite["#anger"] = "#admiration"
opposite[2] = 4
opposite["#surprise"] = "#fear"
opposite[5] = 7
opposite["#fear"] = "#surprise"
opposite[7] = 5
opposite["#interest"] = "#disgust"
opposite[6] = 3
opposite["#disgust"] = "#interest"
opposite[3] = 6

In [7]:
negatives = []
def normal_algo(sen):
    NEGATION_ADVERBS = ["no", "without", "nil","not", "n't", "never", "none", "neith", "nor", "non"]
    NEGATION_VERBS = ["deny", "reject", "refuse", "subside", "retract", "non"]
    CONJUCTION_WORDS = ["for", "and", "nor", "but", "or", "yet", "so"]
    sen = sen.lower()
    sen,removed_user_cnt = remove_count_user_mentions(sen)
    sen,removed_url_cnt = remove_count_urls(sen)
    sen,removed_hashtag_cnt = remove_count_hashtags(sen)
    #print(sen)
    tokens = word_tokenize(sen)
    lem = [wordnet_lemmatizer.lemmatize(t,get_pos(t)) for t in tokens]
    #print(lem)
    lem_lookup = {}
    for i in range(len(tokens)):
        lem_lookup[tokens[i]]=lem[i]
    mark_neg = {}
    nflag = False
    for t in lem:
        if(t[0] in punct or t in CONJUCTION_WORDS):
            nflag=False
        if(nflag==True):
            mark_neg[t]=1
            negatives.append(t)
        if(t in NEGATION_ADVERBS or t in NEGATION_VERBS):
            nflag=True
    tag1 = pos_tag(tokens)
    #print(tag1)
    tokens.clear()
    for x in tag1:
        #print(x)
        if(x[1][0] in need):
            tokens.append(x[0])
    val = {}
    #print(tokens)
    ret_str = ""
    for t in tokens:
        t=lem_lookup[t]
        ret_str+=t
        ret_str+=" "
        """
        if(t in senticnet):
            x = senticnet[t][4]
            #print(t)
            if(t in mark_neg):
                #print(t)
                x=opposite[x]
                #print(t,x)
            if(x in val):
                val[x]+=1
            else:
                val[x]=1
        """
    #print(mark_neg)
    return ret_str

In [8]:
analysed = [normal_algo(txt) for txt in df['Text']]
negatives = set(negatives)
qmark = []
exmark = []
f=0
for txt in df['Text']:
    f=1
    for lt in txt:
        if(lt=='?'):
            qmark.append(1)
            f=0
            break
    if(f==1):
        qmark.append(0)
    f=1
    for lt in txt:
        if(lt=='!'):
            exmark.append(1)
            f=0
            break
    if(f==1):
        exmark.append(0)
print(len(qmark))
print(len(exmark))

8501
8501


In [9]:
print(pd.Series(qmark).value_counts())
print(pd.Series(exmark).value_counts())

0    7425
1    1076
dtype: int64
0    6043
1    2458
dtype: int64


In [10]:
df['Analysed'] = analysed
df['qmark'] = qmark
df['exmark'] = exmark
df.head(10)

,Text,Joy,Sadness,Anger,Disgust,Admiration,Surprise,Interest,Fear,Analysed,qmark,exmark
0,So much for sleeping in.,0,0,0,0,0,0,0,1,so much sleep,0,0
1,College days are loooong days.. 3 more hours,0,1,0,0,0,0,1,0,college day be loooong day more hour,0,0
2,@daihard I'm headed to Kentucky this time. Nev...,0,0,0,0,0,0,1,0,i m head kentucky time never be be fun gqz,1,1
3,hella tired.. where is gilbert for the usual b...,0,0,0,0,0,0,1,0,hella tire be gilbert usual basketball talk,1,1
4,Not as dry this morning as would have liked l...,0,1,0,1,0,0,0,0,not dry morning a have like lot moisture dune ...,0,1
5,@lil_laura_loo Really? I think we have some! I...,0,1,0,0,0,0,0,1,laura loo really i think have i ve take pirite...,1,1
6,@Do512_Kristin it's a good thing they give you...,0,0,0,1,0,0,0,0,kristin s good thing give xanax something i ba...,0,0
7,"PB&amp;J, Owl City, and boredom.",0,1,0,1,0,0,0,0,pb amp j owl city boredom,0,0
8,So it's Saturday again &amp; what do I do..? W...,0,1,0,0,0,0,0,0,s saturday again amp do i do work again course,1,0
9,"trying to relax and watch Nascar, Difficult 'c...",0,1,0,1,0,0,0,0,try relax watch nascar difficult cause child d...,0,0


In [11]:
def get_senticnet(word,em):
    em = '#'+em.lower()
    if(senticnet[word][4]==em or senticnet[word][5]==em):
        return 1
    else:
        return 0

In [12]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
X = df[['Analysed','qmark','exmark']]
Y = df[['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']]

In [13]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df['Analysed'])
print(len(vectorizer.vocabulary_))

10952


In [14]:
kf = KFold(n_splits = 10)
kf.get_n_splits(X)
print(kf)

KFold(n_splits=10, random_state=None, shuffle=False)


In [15]:
def evaluation(score_list,predict_score_list):
    filter_corr = []
    exmatch = 0
    atleast1 = 0
    md1 = 0
    one_f = 0
    more_f = 0
    zero_f = 0
    sm = 0
    sdensity = 0
    hammval = 0
    test_len = len(predict_score_list[0])
    for j in range(test_len):
        cnt=0
        for i in range(8):
            hammval+=(score_list[i][j] ^ int(predict_score_list[i][j]))
            if(score_list[i][j]==1):
                cnt+=1
                sm+=1
        sdensity+=cnt/8
        if(cnt==0):
            zero_f+=1
        if(cnt==1):
            one_f+=1
        if(cnt>1):
            more_f+=1
        for i in range(8):
            mf = True
            if(int(predict_score_list[i][j])!=score_list[i][j]):
                mf=False
                break
        if(mf==True):
            exmatch+=1
            filter_corr.append(j)
        for i in range(8):
            if(int(predict_score_list[i][j])==score_list[i][j] and score_list[i][j]==1):
                atleast1+=1
                break
        mf = False
        for i in range(8):
            if(int(predict_score_list[i][j])==score_list[i][j] and score_list[i][j]==1):
                if(mf==True):
                    md1+=1
                    filter_corr.append(j)
                    break
                mf=True
    #print("Label Cardinality: "+ str(sm/test_len))
    #print("Label Density: "+ str(sdensity/test_len))
    print("Hamming Loss: "+str(hammval/(test_len*8)))
    hamlos = hammval/(test_len*8)
    print("Exact Prediction: "+str(exmatch/test_len))
    sub_accu = exmatch/test_len
    #print("At least one label predicted: "+str(atleast1/(test_len-zero_f)))
    #print("More than one label predicted: "+str(md1/more_f))
    tp_sum = 0
    fp_sum = 0
    fn_sum = 0
    macro_preci = 0
    macro_recall = 0
    macro_f1 = 0
    for i in range(len(score_list)):
        tmp = confusion_matrix(score_list[i],predict_score_list[i])
        tp_sum+=tmp[0][0]
        fp_sum+=tmp[0][1]
        fn_sum+=tmp[1][0]
        macro_preci_tmp=tmp[0][0]/(tmp[0][0]+tmp[0][1])
        macro_recall_tmp=tmp[0][0]/(tmp[0][0]+tmp[1][0])
        macro_f1 += ((2*macro_preci_tmp*macro_recall_tmp)/(macro_preci_tmp+macro_recall_tmp))
        macro_preci+=macro_preci_tmp
        macro_recall+=macro_recall_tmp
        #print(macro_f1)
    micro_preci = tp_sum/(tp_sum+fp_sum)
    micro_recall = tp_sum/(tp_sum+fn_sum)
    micro_f1 = (2*micro_preci*micro_recall)/(micro_preci+micro_recall)
    macro_preci/=8
    macro_recall/=8
    macro_f1/=8
    #print(micro_preci,micro_recall,micro_f1)
    #print(macro_preci,macro_recall,macro_f1)
    print("Macro F-Score: "+str(macro_f1))
    print("Micro F-Score: "+str(micro_f1))
    col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
    tmp = 0
    for i in range(len(score_list)):
        score = accuracy_score(score_list[i],predict_score_list[i]) 
        #print(col_names[i]+" accuracy: "+str(score))
        tmp += score
    print("Average Accuracy: " + str(tmp/8))
    avg_accu = tmp/8
    return (hamlos,sub_accu,macro_f1,micro_f1,avg_accu)

In [18]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
hamm_score = []
subset_accu = []
macro_f1 = []
micro_f1 = []
avg_accu = []
cnt = 1
for train_index,test_index in kf.split(X):
    x_train,x_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = Y.iloc[train_index].values.tolist(),Y.iloc[test_index].values.tolist()
    print("k_fold validation: " + str(cnt))
    cnt+=1
    x_train_analysed = x_train['Analysed'].tolist()
    x_train_qmark = x_train['qmark'].tolist()
    x_train_exmark = x_train['exmark'].tolist()
    x_test_analysed = x_test['Analysed'].tolist()
    x_test_qmark = x_test['qmark'].tolist()
    x_test_exmark = x_test['exmark'].tolist()
    pre = {}
    for sen in x_train_analysed:
        tok = word_tokenize(sen)
        for t in tok:
            pre[t]=1
    for sen in x_test_analysed:
        tok = word_tokenize(sen)
        for t in tok:
            if(t in pre):
                continue
            else:
                if(t in senticnet):
                    x_train_analysed.append(t)
                    x_train_qmark.append(0)
                    x_train_exmark.append(0)
                    tmp_list = []
                    for cl in col_names:
                        tmp_list.append(get_senticnet(t,cl))
                    y_train.append(tmp_list)
    for word in negatives:
        if(word in senticnet):
            x_train_analysed.append("not "+word)
            x_train_qmark.append(0)
            x_train_exmark.append(0)
            tmp_list = []
            for cl in col_names:
                tmp_list.append(get_senticnet(word,cl))
            tmp_list2 = []
            for i in range(8):
                tmp_list2.append(0)
            for i in range(8):
                if(tmp_list[i]==1):
                    tmp_list2[opposite[i]] = 1
            y_train.append(tmp_list2)
    x_train_analysed_vec = vectorizer.transform(x_train_analysed)
    x_test_analysed_vec = vectorizer.transform(x_test_analysed)
    tmp = sparse.hstack((x_train_analysed_vec,np.array(x_train_qmark)[:,None]))
    x_train = sparse.hstack((tmp,np.array(x_train_exmark)[:,None]))
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    tmp = sparse.hstack((x_test_analysed_vec,np.array(x_test_qmark)[:,None]))
    x_test = sparse.hstack((tmp,np.array(x_test_exmark)[:,None]))
    
    print(x_train.shape,x_test.shape)
    print(y_train.shape,y_test.shape)
    
    base = RandomForestClassifier()
    chain_rfc = ClassifierChain(base,order = 'random',random_state=0)
    chain_rfc.fit(x_train,y_train)
    y_pred = chain_rfc.predict(x_test)
    
    y_pred_val = y_pred
    y_test_val = np.array(y_test)
    
    score_list = y_test_val.T.tolist()
    predict_score_list = y_pred_val.T.tolist()
    
    ret = evaluation(score_list,predict_score_list)
    hamm_score.append(ret[0])
    subset_accu.append(ret[1])
    macro_f1.append(ret[2])
    micro_f1.append(ret[3])
    avg_accu.append(ret[4])
    print('\n')
print('Final Result: ')
print('Average Hamming Loss: '+str(sum(hamm_score)/len(hamm_score)))
print('Average Subset Accuracy: '+str(sum(subset_accu)/len(subset_accu)))
print('Average Macro F-score: '+str(sum(macro_f1)/len(macro_f1)))
print('Average Micro F-score: '+str(sum(micro_f1)/len(micro_f1)))
print('Average of Average Accuracy: '+str(sum(avg_accu)/len(avg_accu)))

k_fold validation: 1
(9294, 10954) (851, 10954)
(9294, 8) (851, 8)
Hamming Loss: 0.154524089306698
Exact Prediction: 0.2796709753231492
Macro F-Score: 0.8923793813785309
Micro F-Score: 0.9093103448275863
Average Accuracy: 0.8454759106933021


k_fold validation: 2
(9286, 10954) (850, 10954)
(9286, 8) (850, 8)
Hamming Loss: 0.1652941176470588
Exact Prediction: 0.27176470588235296
Macro F-Score: 0.887411801989982
Micro F-Score: 0.9030533034328101
Average Accuracy: 0.8347058823529411


k_fold validation: 3
(9292, 10954) (850, 10954)
(9292, 8) (850, 8)
Hamming Loss: 0.1648529411764706
Exact Prediction: 0.24588235294117647
Macro F-Score: 0.8873647005009703
Micro F-Score: 0.9027838001907901
Average Accuracy: 0.8351470588235295


k_fold validation: 4
(9344, 10954) (850, 10954)
(9344, 8) (850, 8)
Hamming Loss: 0.16058823529411764
Exact Prediction: 0.26
Macro F-Score: 0.8896274095452457
Micro F-Score: 0.905634289664708
Average Accuracy: 0.8394117647058824


k_fold validation: 5
(9315, 10954) (85

In [19]:
labels = [i+1for i in range(10)]
labels.append('average')
hamm_score.append(sum(hamm_score)/len(hamm_score))
subset_accu.append(sum(subset_accu)/len(subset_accu))
macro_f1.append(sum(macro_f1)/len(macro_f1))
micro_f1.append(sum(micro_f1)/len(micro_f1))
avg_accu.append(sum(avg_accu)/len(avg_accu))

In [20]:
df_rfc = pd.DataFrame(list(zip(labels,hamm_score,subset_accu,macro_f1,micro_f1,avg_accu)),
              columns = ['k-fold','Hamming loss','Subset accuracy','Macro F-score','Micro F-score','Average Accuracy'])
df_rfc

,k-fold,Hamming loss,Subset accuracy,Macro F-score,Micro F-score,Average Accuracy
0,1,0.154524,0.279671,0.892379,0.909310,0.845476
1,2,0.165294,0.271765,0.887412,0.903053,0.834706
2,3,0.164853,0.245882,0.887365,0.902784,0.835147
3,4,0.160588,0.260000,0.889627,0.905634,0.839412
4,5,0.169559,0.240000,0.883792,0.900902,0.830441
5,6,0.163824,0.262353,0.886599,0.903816,0.836176
6,7,0.157353,0.254118,0.892302,0.907487,0.842647
7,8,0.158529,0.262353,0.892006,0.907277,0.841471
8,9,0.165882,0.284706,0.885834,0.902049,0.834118
9,10,0.161029,0.275294,0.889949,0.905252,0.838971


In [21]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
hamm_score = []
subset_accu = []
macro_f1 = []
micro_f1 = []
avg_accu = []
cnt = 1
for train_index,test_index in kf.split(X):
    x_train,x_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = Y.iloc[train_index].values.tolist(),Y.iloc[test_index].values.tolist()
    print("k_fold validation: " + str(cnt))
    cnt+=1
    x_train_analysed = x_train['Analysed'].tolist()
    x_train_qmark = x_train['qmark'].tolist()
    x_train_exmark = x_train['exmark'].tolist()
    x_test_analysed = x_test['Analysed'].tolist()
    x_test_qmark = x_test['qmark'].tolist()
    x_test_exmark = x_test['exmark'].tolist()
    pre = {}
    for sen in x_train_analysed:
        tok = word_tokenize(sen)
        for t in tok:
            pre[t]=1
    for sen in x_test_analysed:
        tok = word_tokenize(sen)
        for t in tok:
            if(t in pre):
                continue
            else:
                if(t in senticnet):
                    x_train_analysed.append(t)
                    x_train_qmark.append(0)
                    x_train_exmark.append(0)
                    tmp_list = []
                    for cl in col_names:
                        tmp_list.append(get_senticnet(t,cl))
                    y_train.append(tmp_list)
    for word in negatives:
        if(word in senticnet):
            x_train_analysed.append("not "+word)
            x_train_qmark.append(0)
            x_train_exmark.append(0)
            tmp_list = []
            for cl in col_names:
                tmp_list.append(get_senticnet(word,cl))
            tmp_list2 = []
            for i in range(8):
                tmp_list2.append(0)
            for i in range(8):
                if(tmp_list[i]==1):
                    tmp_list2[opposite[i]] = 1
            y_train.append(tmp_list2)
    x_train_analysed_vec = vectorizer.transform(x_train_analysed)
    x_test_analysed_vec = vectorizer.transform(x_test_analysed)
    tmp = sparse.hstack((x_train_analysed_vec,np.array(x_train_qmark)[:,None]))
    x_train = sparse.hstack((tmp,np.array(x_train_exmark)[:,None]))
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    tmp = sparse.hstack((x_test_analysed_vec,np.array(x_test_qmark)[:,None]))
    x_test = sparse.hstack((tmp,np.array(x_test_exmark)[:,None]))
    
    print(x_train.shape,x_test.shape)
    print(y_train.shape,y_test.shape)
    
    base = SVC()
    chain_rfc = ClassifierChain(base,order = 'random',random_state=0)
    chain_rfc.fit(x_train,y_train)
    y_pred = chain_rfc.predict(x_test)
    
    y_pred_val = y_pred
    y_test_val = np.array(y_test)
    
    score_list = y_test_val.T.tolist()
    predict_score_list = y_pred_val.T.tolist()
    
    ret = evaluation(score_list,predict_score_list)
    hamm_score.append(ret[0])
    subset_accu.append(ret[1])
    macro_f1.append(ret[2])
    micro_f1.append(ret[3])
    avg_accu.append(ret[4])
    print('\n')
print('Final Result: ')
print('Average Hamming Loss: '+str(sum(hamm_score)/len(hamm_score)))
print('Average Subset Accuracy: '+str(sum(subset_accu)/len(subset_accu)))
print('Average Macro F-score: '+str(sum(macro_f1)/len(macro_f1)))
print('Average Micro F-score: '+str(sum(micro_f1)/len(micro_f1)))
print('Average of Average Accuracy: '+str(sum(avg_accu)/len(avg_accu)))

k_fold validation: 1
(9294, 10954) (851, 10954)
(9294, 8) (851, 8)
Hamming Loss: 0.1636310223266745
Exact Prediction: 0.28319623971797886
Macro F-Score: 0.8772918105574208
Micro F-Score: 0.9029109290569983
Average Accuracy: 0.8363689776733254


k_fold validation: 2
(9286, 10954) (850, 10954)
(9286, 8) (850, 8)
Hamming Loss: 0.17558823529411766
Exact Prediction: 0.29411764705882354
Macro F-Score: 0.8724714753968911
Micro F-Score: 0.8956293706293706
Average Accuracy: 0.8244117647058823


k_fold validation: 3
(9292, 10954) (850, 10954)
(9292, 8) (850, 8)
Hamming Loss: 0.16661764705882354
Exact Prediction: 0.2788235294117647
Macro F-Score: 0.8802208610327236
Micro F-Score: 0.9006576063130207
Average Accuracy: 0.8333823529411764


k_fold validation: 4
(9344, 10954) (850, 10954)
(9344, 8) (850, 8)
Hamming Loss: 0.1676470588235294
Exact Prediction: 0.2964705882352941
Macro F-Score: 0.8778370737724617
Micro F-Score: 0.9002100840336134
Average Accuracy: 0.8323529411764706


k_fold validation: 5

In [22]:
labels = [i+1for i in range(10)]
labels.append('average')
hamm_score.append(sum(hamm_score)/len(hamm_score))
subset_accu.append(sum(subset_accu)/len(subset_accu))
macro_f1.append(sum(macro_f1)/len(macro_f1))
micro_f1.append(sum(micro_f1)/len(micro_f1))
avg_accu.append(sum(avg_accu)/len(avg_accu))

In [23]:
df_svc = pd.DataFrame(list(zip(labels,hamm_score,subset_accu,macro_f1,micro_f1,avg_accu)),
              columns = ['k-fold','Hamming loss','Subset accuracy','Macro F-score','Micro F-score','Average Accuracy'])
df_svc

,k-fold,Hamming loss,Subset accuracy,Macro F-score,Micro F-score,Average Accuracy
0,1,0.163631,0.283196,0.877292,0.902911,0.836369
1,2,0.175588,0.294118,0.872471,0.895629,0.824412
2,3,0.166618,0.278824,0.880221,0.900658,0.833382
3,4,0.167647,0.296471,0.877837,0.900210,0.832353
4,5,0.176912,0.256471,0.870158,0.895035,0.823088
5,6,0.171324,0.271765,0.874439,0.898173,0.828676
6,7,0.168824,0.264706,0.875728,0.899404,0.831176
7,8,0.168529,0.298824,0.876869,0.900017,0.831471
8,9,0.172794,0.282353,0.873134,0.897011,0.827206
9,10,0.176471,0.281176,0.868259,0.894921,0.823529
